In [35]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import json
import time
import random
import datetime as dt
from datetime import timedelta

from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import RandomizedLasso
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import sklearn.cross_validation
from sklearn import discriminant_analysis
from sklearn import naive_bayes
from sklearn import tree
from sklearn import cluster
from sklearn import metrics
from sklearn import cross_validation
from sklearn import svm

import xgboost as xgb

path="/Users/jnoxon/Dropbox/python/"

In [11]:
from IPython.core.display import HTML
def css_styling():
    styles = open(path + "custom.css", "r").read()
    return HTML(styles)
css_styling()

In [19]:
iris = pd.read_csv("../iris.data", header=None)
y = iris[4]
iris = iris.drop([4], 1)
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(iris, y, test_size = .20, random_state=42)

In [29]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
Y = df['quality'].values
df = df.drop('quality',1)
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(df, Y, test_size = .20, random_state=42)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:590)>

In [ ]:
cancer = pd.read_csv(path + 'Oracle Development/breast_cancer.csv')
#http://www.is.umk.pl/projects/datasets.html
y = np.array(cancer['2.1'])
x = np.array(cancer.drop(['2.1', '1.3'], 1))
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(x, y, test_size = .20, random_state=42)

In [14]:
votes = pd.read_csv(path + '/Oracle Development/votes.csv')
votes = votes.replace('?', float('NaN'))
votes = votes.dropna()
votes = votes.reset_index(drop=True)
votes['0'] = votes.replace('D', 1)
votes['0'] = votes.replace('R', 0)
y = np.array(votes['0'])
votes = votes.drop('0', axis = 1)
votes = votes.replace('y', 1)
votes = votes.replace('n', 0)
x = np.array(votes)
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(np.array(x), np.array(y), test_size = .20, random_state=42)

In [47]:
def evaluate(grid=False):
    iris = pd.read_csv("../iris.data", header=None)
    y = iris[4]
    iris = iris.drop([4], 1)
    print "Iris:"
    
    problem = classifier(iris,y, grid=grid)
    problem.predict()
    
    
    
#     df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
#     Y = df['quality'].values
#     df = df.drop('quality',1)
#     print"\nWine Quality:"
    
#     problem = classifier(df, Y, grid=grid)
#     problem.predict()
    
    
    
    cancer = pd.read_csv(path + 'Oracle Development/breast_cancer.csv')
    #http://www.is.umk.pl/projects/datasets.html
    y = np.array(cancer['2.1'])
    x = np.array(cancer.drop(['2.1', '1.3'], 1))
    print "\nBreast Cancer:"
    
    problem = classifier(x, y, grid=grid)
    problem.predict()
    
    
    
    votes = pd.read_csv(path + '/Oracle Development/votes.csv')
    votes = votes.replace('?', float('NaN'))
    votes = votes.dropna()
    votes = votes.reset_index(drop=True)
    votes['0'] = votes.replace('D', 1)
    votes['0'] = votes.replace('R', 0)
    y = np.array(votes['0'])
    votes = votes.drop('0', axis = 1)
    votes = votes.replace('y', 1)
    votes = votes.replace('n', 0)
    x = np.array(votes)
    print "\nVotes:"
    
    problem = classifier(x, y, grid=grid)
    problem.predict()


In [52]:
class classifier():
    def __init__(self, x, y, grid=False):
        self.grid = grid
        if type(y[0]) != int:
            y, self.mapping = self.categorize(y)
        self.x_train, self.x_test, self.y_train, self.y_test = sklearn.cross_validation.train_test_split\
            (np.array(x), np.array(y, dtype = int), test_size = .20, random_state=42)
    
    def accuracy(self, predictions):
        count = 0
        for i in xrange(len(predictions)):
            if predictions[i] == self.y_test[i]:
                count += 1
        return count/float(len(predictions))
    
    def categorize(self, labels):
        mapping = {}
        labels = pd.Series(labels)
        for i,v in enumerate(list(set(labels))):
            labels = labels.replace(v, i)
            mapping[i] = v
        return labels, mapping
    
    def xgboost(self):
        dtrain = xgb.DMatrix(self.x_train, label=self.y_train)
        dtest = xgb.DMatrix(self.x_test)
        param = {'bst:max_depth':50, 'bst:eta':1, 'silent':1,'objective':'multi:softmax'}
        param['num_class'] = self.x_train.shape[1]
#         evallist = [(dtest,'eval'), (dtrain,'train')]
        num_round = 10
        bst = xgb.train(param, dtrain, num_round, verbose_eval=False)
        preds = bst.predict(dtest)
#         for i in xrange(len(labels)):
#             print labels[i], self.y_test[i]
        return preds
    
    def gradient_booster(self, n_estimators=100):
        if self.grid == False:
            booster = ensemble.GradientBoostingClassifier(n_estimators = 200)
            booster = booster.fit(self.x_train, self.y_train)
            booster_predictions = booster.predict(self.x_test)
            return booster_predictions
        else:
            for n_estimators in xrange(100, 1000, 300):
                for max_depth in xrange(2,15,3):
                    booster = ensemble.GradientBoostingClassifier\
                    (n_estimators = n_estimators, max_depth = max_depth)
                    booster = booster.fit(self.x_train, self.y_train)
                    booster_predictions = booster.predict(self.x_test)
                    max_accuracy = -1
                    print self.accuracy(booster_predictions)
                    if self.accuracy(booster_predictions) > max_accuracy:
                        max_accuracy = self.accuracy(booster_predictions)
                        max_predictions = booster_predictions
                        optimal_parameters = [n_estimators, max_depth]
            print("Gradient booster: ", optimal_parameters)
            return max_predictions
                
    
    def random_forest(self, n_estimators=100):
        if self.grid == False:
            forest = ensemble.RandomForestClassifier(n_estimators)
            forest = forest.fit(self.x_train, self.y_train)
            forest_predictions = forest.predict(self.x_test)
            return forest_predictions
        else:
            feat = x_train.shape[1]
            for n_estimators in xrange(100, 1000, 200):
                for proportion in xrange(int(.3*feat), int(.6*feat), int(.03*feat)+1):
                    for criterion in ['gini', 'entropy']:
                        forest = ensemble.RandomForestClassifier(n_estimators = n_estimators, criterion = criterion, max_features = proportion)
                        forest = forest.fit(self.x_train, self.y_train)
                        forest_predictions = forest.predict(self.x_test)
                        max_accuracy = -1
                        print self.accuracy(forest_predictions)
                        if self.accuracy(forest_predictions) > max_accuracy:
                            max_accuracy = self.accuracy(forest_predictions)
                            max_predictions = forest_predictions
                            optimal_parameters = [n_estimators, proportion, criterion]
            print["Random forest: ", optimal_parameters]
            return max_predictions
                    
    
    def extremely_random_forest(self, n_estimators=100):
        erf = ensemble.ExtraTreesClassifier(n_estimators)
        erf = erf.fit(self.x_train, self.y_train)
        erf_predictions = erf.predict(self.x_test)
        return erf_predictions
    
    def logistic_regression(self):
        logistic = linear_model.LogisticRegression(penalty="l2")#class_weight='auto'
        logistic.fit(self.x_train, self.y_train)
        logistic_predictions = logistic.predict(self.x_test)
        return logistic_predictions
    
    def lda(self):
        lda = discriminant_analysis.LinearDiscriminantAnalysis()
        lda.fit(self.x_train, self.y_train)
        lda_predictions = lda.predict(self.x_test)
        return lda_predictions
    
    def qda(self):
        qda = discriminant_analysis.QuadraticDiscriminantAnalysis()
        qda.fit(self.x_train, self.y_train)
        qda_predictions = qda.predict(self.x_test)
        return qda_predictions
    
    def nb_gaussian(self):
        nb = naive_bayes.GaussianNB()
        nb.fit(self.x_train, self.y_train)
        nb_predictions = nb.predict(self.x_test)
        return nb_predictions
    
    def nb_multinomial(self):
        nb = naive_bayes.MultinomialNB()
        nb.fit(self.x_train, self.y_train)
        nb_predictions = nb.predict(self.x_test)
        return nb_predictions
    
    def svc(self):
        support_vm = svm.SVC()
        support_vm.fit(self.x_train, self.y_train)
        svm_predictions = support_vm.predict(self.x_test)
        return svm_predictions
    
    
    def blend(self, *args):
        n = len(args)
        blended_predictions = []
        for i in xrange(len(args[0])):
            counts = {}
            for j in xrange(n):
                try:
                    counts[args[j][i]] += 1
                except KeyError:
                    counts[args[j][i]] = 1
            maximus = -float('inf')
            for index, value in counts.iteritems():
                if value > maximus:
                    max_value = index
                    maximus = value
            blended_predictions.append(max_value)
        return blended_predictions

    def predict(self):
        xgboost_predictions = self.xgboost()
        booster_predictions = self.gradient_booster()
        forest_predictions = self.random_forest()
        erf_predictions = self.extremely_random_forest()
        logistic_predictions = self.logistic_regression()
        lda_predictions = self.lda()
        qda_predictions = self.qda()
        nb_g = self.nb_gaussian()
        nb_m = self.nb_multinomial()
        svm_predictions = self.svc()
        blended_predictions = self.blend(xgboost_predictions, booster_predictions, forest_predictions, erf_predictions,\
                  logistic_predictions, lda_predictions, qda_predictions, nb_g, nb_m, svm_predictions)
        
        models = ['xgboost_predictions','booster_predictions', 'forest_predictions', 'erf_predictions',\
                  'logistic_predictions', 'lda_predictions', 'qda_predictions', \
                  'naive_bayes_gaussian', 'naive_bayes_multinomial', 'svm_predictions', 'blended_predictions']
        for i, v in enumerate([xgboost_predictions, booster_predictions, forest_predictions, erf_predictions,\
                  logistic_predictions, lda_predictions, qda_predictions,\
                 nb_g, nb_m, svm_predictions, blended_predictions]):
            
            print self.accuracy(v), models[i]

In [53]:
evaluate(grid=True)

Iris:
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
('Gradient booster: ', [700, 14])
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
['Random forest: ', [900, 1, 'entropy']]
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
['Random forest: ', [900, 1, 'entropy']]
1.0 xgboost_predictions
1.0 booster_predictions
1.0 forest_predictions
1.0 erf_predictions
1.0 logistic_predictions
1.0 lda_predictions
0.966666666667 qda_predictions
1.0 naive_bayes_gaussian
0.9 naive_bayes_multinomial
1.0 svm_predictions
1.0 blended_predictions

Breast Cancer:
0.971428571429
0.971428571429
0.942857142857
0.95
0.95
0.978571428571
0.971428571429
0.942857142857
0.957142857143
0.95
0.971428571429
0.971428571429
0.942857142857
0.95
0.95
('Gradient booster: ', [700, 14])
0.978571428571
0.978571428571
0.978571428571
0.978571428571
0.978571428571
0.971428571429
0.978571428571
0.978571428571
0.978571428571
0.978571428571
['Random forest: ', [900, 1, 'entropy']]
0.978571428571
0.978571428571
0.978571428571
0.97142857142

In [39]:
evaluate()

Iris:
1.0 xgboost_predictions
1.0 booster_predictions
1.0 forest_predictions
1.0 erf_predictions
1.0 logistic_predictions
1.0 lda_predictions
0.966666666667 qda_predictions
1.0 naive_bayes_gaussian
0.9 naive_bayes_multinomial
1.0 blended_predictions

Wine Quality:
0.625 xgboost_predictions
0.653125 booster_predictions
0.678125 forest_predictions
0.68125 erf_predictions
0.56875 logistic_predictions
0.559375 lda_predictions
0.55625 qda_predictions
0.55 naive_bayes_gaussian
0.475 naive_bayes_multinomial
0.65 blended_predictions

Breast Cancer:
0.964285714286 xgboost_predictions
0.978571428571 booster_predictions
0.971428571429 forest_predictions
0.971428571429 erf_predictions
0.635714285714 logistic_predictions
0.95 lda_predictions
0.971428571429 qda_predictions
0.842857142857 naive_bayes_gaussian
0.942857142857 naive_bayes_multinomial
0.964285714286 blended_predictions

Votes:
0.936170212766 xgboost_predictions
0.936170212766 booster_predictions
0.957446808511 forest_predictions
0.978723

In [46]:
iris = pd.read_csv("../iris.data", header=None)
y = iris[4]
iris = iris.drop([4], 1)
print "Iris:"
problem = classifier(iris,y)
problem.predict()

Iris:
1.0 booster_predictions
1.0 forest_predictions
1.0 erf_predictions
1.0 logistic_predictions
1.0 lda_predictions
0.966666666667 qda_predictions
1.0 naive_bayes_gaussian
0.9 naive_bayes_multinomial
